In [59]:
import pandas as pd
import numpy as np
from functools import reduce
import re

balance_path = r"E:\RR\balance\0408.xlsx"
revenue_path = r"E:\RR\revenue\revenue1.xlsx"
result1 = r"E:\RR\result1.xlsx"
GLoffset = r"E:\RR\GLoff.xlsx"

t_gl = pd.read_excel(balance_path, dtype={'项目名称': str, '项目编码': str, '客户编码': str}).fillna(0)[["科目编码", "科目名称", "项目名称", "项目编码", "客户编码", "客户名称", "方向_", "期末余额金额"]]
t_gl['期末余额金额'] = np.where(t_gl['方向_']=="借", -1 * t_gl['期末余额金额'], t_gl['期末余额金额'])
t_gl = t_gl.loc[(t_gl["科目编码"]!=1221996)&(t_gl["项目编码"]!=0)]
t_gl_0 = t_gl.copy()########ORIGIN
t_gl

,科目编码,科目名称,项目名称,项目编码,客户编码,客户名称,方向_,期末余额金额
698,2203001,人民币,0000000,00001078,00000000,通用客户,贷,375.01
700,2203001,人民币,2012166,00000566,00000012,广州新盟物流有限公司,贷,1716.90
702,2203001,人民币,2012167,00000567,00000017,深圳水立方健康俱乐部,借,-6000.00
704,2203001,人民币,2012208,00000608,00000023,深圳市中科远东创业投资,贷,12000.00
705,2203001,人民币,2012209,00000609,00000023,深圳市中科远东创业投资,贷,12000.00
...,...,...,...,...,...,...,...,...
12849,2241028,代收支费用,gz20201343,00020851,sz013083,深圳市新桥新二股份合作公司,贷,135000.00
12851,2241028,代收支费用,gz20212828,00030322,sz013549,北京淳一航空科技有限公司,贷,3300.00
12853,2241028,代收支费用,gz20214681,00032175,sz013552,贾漪,贷,29000.00
12855,2241028,代收支费用,gz20204682,00024190,sz013866,任光驰,贷,27840.00


In [60]:
def settle(id):#冲预收
    df_rec = t_gl.loc[(t_gl['项目名称']==t_rr.loc[id].受案号)].sort_values(by='期末余额金额')# ascending
    sett = -1*(t_rr.loc[id].收入+t_rr.loc[id].增值税)+0.000001# the offset_amount = revenue + VAT
    ## float type always have issues when comparing to each other, for quick use I skip the details
    if df_rec['期末余额金额'].sum() + sett >= 0:# exploiting list as a queue to do the Fifo job
        Q = df_rec["期末余额金额"].tolist()
        Le = len(Q)
        while Q[0] + sett < 0:
            sett = sett + Q[0]
            Q.pop(0)
        
        Q[0] = Q[0] + sett
        Q = [0*i for i in range(Le-len(Q))]+Q
        df_rec["期末余额金额"] = np.array(Q)# 1. Update the gl records after every settlement transaction
                                           # np.array is almost the default method of updating a column of values in pandas
        t_rr.loc[id,'sett'] = 1.0# 2. Mark the line items that has been successfully settled
                                 # Always update cell value using "df.loc[index, column_name]= value" style
    else:
        print("Error, please check")
    for id in df_rec.index.values:
        t_gl.loc[id, "期末余额金额"] = df_rec.loc[id, "期末余额金额"]
        print("Settled, please check the result")

def CaseCode(string):#调整案号
    def CasePrefix(string):
        if "医疗" in string:
            return "yl"
        elif "深仲涉外" in string:
            return "sw"
        elif "深仲" in string:
            return "sz"
        elif "深国仲" in string:
            return "gz"
        else:
            return string
    def CaseNum(string):
        if len(re.findall(r"\d+.?\d*",string)) == 0:
            return ""
        else:
            numbers = str(reduce(lambda x, y: x + y, re.findall(r"\d+\.?\d*",string)))
            if len(numbers) < 8:
                numbers = numbers[0:4]+"0"*(8-len(numbers))+numbers[4:]
                return numbers
            else:
                return numbers[0:8]
            
    return CasePrefix(string) + str(CaseNum(string))

In [61]:
t_rr = pd.read_excel(revenue_path, sheet_name="Sheet1")[["受案号", "收入", "增值税"]]
    #[["受案号", "申请方收入", "申请方增值税", "被申请方收入", "被申请方增值税", "收入", "增值税", "申请方", "被申请方"]]
# t_rr.filter()
t_rr['sett']=np.NaN
t_rr["受案号"] = t_rr["受案号"].apply(CaseCode)
t_rr.fillna(0, inplace=True)#.loc[19]
t_rr


,受案号,收入,增值税,sett
0,统建楼租金,37894.40,1894.72,0.0
1,汇峰苑租金,9958.10,497.90,0.0
2,gz20217007,5711.43,285.57,0.0
3,gz20220103,21600.00,1080.00,0.0
4,gz20216965,7714.29,385.71,0.0
...,...,...,...,...
513,gz20214663,100580.11,1166.79,0.0
514,gz20215118,91548.20,572.80,0.0
515,gz20215957,20000.00,0.00,0.0
516,gz20216155,81577.50,0.00,0.0


In [62]:
#test_rec = t_gl.loc[(t_gl['项目名称']==t_rr.loc[19].受案号)].sort_values(by='期末余额金额')
#test_rec['期末余额金额'] = np.array([100.0,200.0])
#test_rec.index#.values
#settle(19)

for id in t_rr.index.values:
    print(id)
    settle(id)

t_gl_1 = t_gl.copy()

0
Error, please check
1
Error, please check
2
Settled, please check the result
3
Settled, please check the result
4
Settled, please check the result
5
Settled, please check the result
Settled, please check the result
6
Settled, please check the result
Settled, please check the result
7
Settled, please check the result
8
Settled, please check the result
9
Settled, please check the result
10
Settled, please check the result
11
Settled, please check the result
12
Settled, please check the result
13
Settled, please check the result
14
Settled, please check the result
15
Settled, please check the result
16
Settled, please check the result
17
Settled, please check the result
18
Settled, please check the result
19
Settled, please check the result
20
Settled, please check the result
21
Settled, please check the result
22
Settled, please check the result
23
Settled, please check the result
Settled, please check the result
24
Settled, please check the result
25
Settled, please check the result
2

In [63]:
#t_gl.loc[t_gl['项目名称']=="gz20204766"]
t_rr
t_rr.to_excel(result1)

In [64]:
t_gl_offset = t_gl_0.copy()
t_gl_offset['g']=t_gl_1['期末余额金额']
t_gl_offset['diff']=t_gl_offset['期末余额金额']-t_gl_offset['g']
t_gl_offset = t_gl_offset.loc[t_gl_offset['diff']!=0].drop(["方向_","期末余额金额","g"],axis=1)
#print(np.sum(t_gl_offset["diff"]),np.sum(t_rr.loc[t_rr['sett']==1.0]['增值税']+t_rr.loc[t_rr['sett']==1.0]['收入']))
t_rr.loc[t_rr['sett']!=1.0]

,受案号,收入,增值税,sett
0,统建楼租金,37894.40,1894.72,0.0
1,汇峰苑租金,9958.10,497.90,0.0
504,DP2018068420180684,35568.69,1749.31,0.0


In [65]:
t_gl_offset.to_excel(GLoffset)